In [ ]:
print(__doc__)

import pandas as pd
import numpy as np
import math

#Exercise 4 - mushroom dataset
df = pd.read_csv('mushrooms.csv')

#1 = edable, 0=poisonous
y2 = df['class']
X2 = df.drop(['class'], axis=1)

X_all = pd.concat((X2, y2.T), axis=1)
X_all = clean_data(X_all)

X_train, X_test, X_prun = separate_data(X_all)

y_train = X_train.iloc[:,-1]
X_train = X_train.iloc[:,:-1]

root = learn(X_train, y_train, impurity_measure='entropy', feature_names=True)

#If you want to run it with pruning
#root = learn(X_train, y_train, impurity_measure='entropy', pruning=X_prun, feature_names=False)

#Small dataset I used to build up the application, don't think this will work now because of the multiple labels
training_data = np.array([
    ['Green', 3, 'sweet', 'Apple'],
    ['Yellow', 3, 'sour','Apple'],
    ['Red', 1, 'sweet', 'Grape'],
    ['Red', 1, 'sweet', 'Grape'],
    ['Yellow', 3, 'sour', 'Lemon'],
    ['Green', 2, 'sour', 'Lime'],
    ['Green', 1, 'sweet', 'Grape'],
    ['Red', 5, 'sweet', 'Melon'],
    ['Yellow', 5, 'sweet', 'Melon'],
])


#Here the user needs to import a dataset, put all the data into X, and all the targets into y
X = training_data[:, :(len(training_data[0])-1)]
y = np.array([training_data[:, 3]])

#To build the tree run this line
#root = learn(X, y, impurity_measure=?, pruning=?, feature_names=?)
#Example of predict
#label predict(['Purple', 5, 'salty'], root)

In [ ]:

def separate_data(data, test_share=0.3, pruning_share=0.1):
    """
    Doc.

    Title: separate_data

    Separates the whole dataset into training, test and pruning set

    :param data: the whole dataset
    :type data: pandas dataframe

    :param test_share: the size of the testset in percentage - default = 0.3
    :type test_share: float

    :param pruning_share: the size of the pruningset in percentage - default = 0.1
    :type pruning_share: float

    :return training_data: trainingset filled with data
    :type training_data: pandas dataframe

    :return test_data: testset filled with data
    :type test_data: pandas dataframe

    :return prun_data: trainingset filled with data
    :type prun_data: pandas dataframe
    """
    num_prun = int(len(data)*pruning_share)
    num_test = int(len(data)*test_share)
    
    test_data = data.iloc[:num_test,:]
    prun_data = data.iloc[num_test:(num_prun+num_test), :]
    training_data = data.iloc[(num_test+num_prun):, :]

    return training_data, test_data, prun_data

In [ ]:

class Tree(object):
    """
    Doc.

    Title: Tree

    At tree class I got from the second answer:
    https://stackoverflow.com/questions/41760856/most-simple-tree-data-structure-in-python-that-can-be-easily-traversed-in-both

    I have done som adjustments, and added a couple of variables

    :param data: is a collection of data and the labels(target), seperated in categories
    :type data: 2D array

    :param split_value: where to split the data in current node(Tree) - deafault = None
    :type split_value: string or number(float, int)

    :param col: the column in the data that is used to split - default = None
    :type col: int

    :param children: holds the children with a key that is True or False based on the value comparison with the split_value,
    answer to the question. And value is the children node - default = None
    :type children: dict

    :param parent: parent to the current node - default = None
    :type parent: Tree
    """ 
    def __init__(self, data, split_value=None, col=None, children=None, parent=None):
        self.data = data
        self.split_value = split_value
        self.col = col
        self.children = children or {}
        self.parent = parent

    def add_child(self, data, key):
        new_child = Tree(data, parent=self)
        self.children.update({key: new_child})
        return new_child

    def is_root(self):
        return self.parent is None

    def is_leaf(self):
        return not self.children

    def __str__(self):
        if self.is_leaf():
            return str(self.data)
        return '{data} [{children}]'.format(data=self.data, children=', '.join(map(str, self.children)))

In [ ]:

def clean_data(data):
    """
    Doc.

    Title: clean_data

    Removes rows with '?' value in it

    :param data: is a collection of data and the labels(target), seperated in categories
    :type data: pandas dataframe

    :return: data, without the rows with '?' in it
    :return type: pandas dataframe
    """
    return data[~(data == '?').any(axis=1)]
    

In [ ]:

def is_number(value):
    """
    Doc.

    Title: is_number

    Validate if "value" is a number or not

    :param value:  validation value
    :type value: string or int

    :return: if value is number or not
    :return type: bool

    """
    try:
        float(value)
        return True
    except ValueError:
        return False

In [ ]:

def value_count(data, col):
    """
    Doc.

    Title: value_count

    Counts the different values in a category

    :param data: is a collection of data and the labels(target), seperated in categories
    :type data: 2D array

    :param col: the column in the data that are supposed to be counted
    :type col: int

    :return: a collection where key is the category value and value is the number of times the key has occurred
    :return type: dict
    """
    count = {}
    for row in data:
        label = row[col]
        if label not in count:
            count[label] = 0
        count[label] += 1
    return count

In [ ]:

def get_label(data):
    """
    Doc.

    Title: get_label

    Gets the most common label in a dataset

    :param data: is a collection of data and the labels(target), seperated in categories
    :type data: 2D array

    :return: the label thet occurred the most times
    :return type: depends on label type, number(int, float) or string
    """
    label_max = 0
    label = None
    count = value_count(data, -1)
    
    for key, value in count.items():
        if(value > label_max):
            label = key
    
    return label

# Exercise 1 & 2

In [ ]:

def entropy(data):
    """
    Doc.

    Title: entropy

    Finds the entropy of a dataset

    :param data: is a collection of data and the labels(target), seperated in categories
    :type data: 2D array

    :return: the entropy
    :return type: float
    """
    label_col = -1
    count = value_count(data, label_col)
    entropy = 0
    #Calculation of entropy
    for key, value in count.items():
        entropy = entropy - (((value/len(data))*(math.log2(value/len(data)))))
    return entropy


In [ ]:

def gini_index(data):
    """
    Doc.

    Title: gini_index

    Finds the gini index of a dataset

    :param data: is a collection of data and the labels(target), seperated in categories
    :type data: 2D array

    :return: the gini index
    :return type: float
    """
    label_col = -1
    count = value_count(data, label_col)
    gini_index = 0
    #Calculation of gini index
    for key, value in count.items():
        gini_index += (value/len(data))*(1-(value/len(data)))
    return gini_index

In [ ]:

def question(split_value, new_value):
    """
    Doc.

    Title: question

    Takes in two values and compare them

    :param split_value: is the value that decides where to split the parentnode
    :type split_value: string or number(float, int)

    :param new_value: decides which child to put the row in after comparing it to split_value
    :type new_value: string or number(float, int)

    :return: the answer to the question
    :return type: bool
    """
    if is_number(split_value):
        return int(new_value) >= int(split_value) 
    else:
        return split_value == new_value

In [ ]:

def split(data, split_value, split_col):
    """
    Doc.

    Title: split

    Splits a dataset into two parts, based on the split_value

    :param data: is a collection of data and the labels(target), seperated in categories
    :type data: 2D array

    :param split_value: is the value that decides where to split the data
    :type split_value: string or number(float, int)

    :param split_col: the column where the split is happening
    :type split_col: int

    :return left_values: the "False" values, after comparing with split_value. Left children
    :return type: 2D array

    :return right_values: the "True" values, after comparing with split_value. Right children
    :return type: 2D array
    """
    left_values, right_values = [],[]
    
    for row in data:    
        if question(split_value, row[split_col]):
            right_values = np.append(right_values, row)
        else:
            left_values = np.append(left_values, row)
    
    #To get it correctly formatted in a 2D np.array, I couldn't find a betterway
    left_values = np.reshape(left_values, (-1, len(data[0])))
    right_values = np.reshape(right_values, (-1, len(data[0])))
    
    
    return left_values, right_values

In [ ]:

def best_info_gain_entropy(data):
    """
    Doc.

    Title: best_info_gain_entropy

    Finds the smallest summarzed entropi in the children after a split. Same as highest information gain

    :param data: is a collection of data and the labels(target), seperated in categories
    :type data: 2D array

    :return split_value: the value to split on from the best information gain
    :type split_value: string or number(float, int)

    :param min_col: the column where the best split_value is
    :type min_col: int

    :return left_values: the "False" values, after comparing with split_value. Left children
    :return type: 2D array

    :return right_values: the "True" values, after comparing with split_value. Right children
    :return type: 2D array
    """
    entropy_parent = entropy(data)
    min_entropy=entropy_parent
    min_col=0
    split_value = None
    min_left_child=[]
    min_right_child=[]
    
    #loops through the columns
    for i in range(len(data[0])-1):
        unique_values = np.unique(data[:, i])
        #loops through the unique values in a column 
        for value in unique_values:
            #do a split on every value
            left_child, right_child = split(data, value, i)
            #calculate the entropy
            entropy_child = (len(left_child)/len(data))*entropy(left_child) + (len(right_child)/len(data))*entropy(right_child)
            
            if entropy_child < min_entropy:
                min_entropy = entropy_child
                min_col = i
                split_value = value
                min_left_child = left_child
                min_right_child = right_child
    
    #Under is the IG, but we don't need it, because it's enough to find the smallest entropy in the children.
    #After this we dosen't need the entropy or the IG anymore because we have the best split_value
    
    #information_gain = entropy_parent - min_entropy
    
    return split_value, min_col, min_left_child, min_right_child

In [ ]:
def best_info_gain_gi(data):
    """

    gi_parent = gini_index(data)
    min_gi=gi_parent
    min_col=0    Doc.

    Title: best_info

    Finds the smallest summarzed gini index in the children after a split. Same as highest information gain

    :param data: is a collection of data and the labels(target), seperated in categories
    :type data: 2D array

    :return split_value: the value to split on from the best information gain
    :type split_value: string or number(float, int)

    :param min_col: the column where the best split_value is
    :type min_col: int

    :return left_values: the "False" values, after comparing with split_value. Left children
    :return type: 2D array

    :return right_values: the "True" values, after comparing with split_value. Right children
    :return type: 2D array
    """
    gi_parent = gini_index(data)
    split_value = None
    min_col=0
    min_left_child = []
    min_right_child = []
    min_gi=gi_parent
    #loops through the columns
    for i in range(len(data[0])-1):
        unique_values = np.unique(data[:, i])
        #loops through the unique values in a column 
        for value in unique_values:
            #do a split on every value
            left_child, right_child = split(data, value, i)
            #calculate the gini index
            gi_child = (len(left_child)/len(data))*gini_index(left_child) + (len(right_child)/len(data))*gini_index(right_child)
            
            if gi_child < min_gi:
                min_gi = gi_child
                min_col = i
                split_value = value
                min_left_child = left_child
                min_right_child = right_child            
    #Under is the IG, but we don't need it, because it's enough to find the smallest gini index in the child.
    #After this we dosen't need the gini index or the IG anymore because enough to have the best split_value
    
    #information_gain = gi_parent - min_gi      
    
    return split_value, min_col, min_left_child, min_right_child

In [ ]:
def build_tree(parent_node, impurity_measure):
    """
    Doc.

    Title: build_tree

    Recursive function for building the tree

    :param parent_node: the current node in the tree
    :type parent_node: Tree

    :param impurity_measure: defines which impurity measure to use. Entorpy or gini index
    :type impurity_measure: string

    :return: no return, the tree makes itself when running this function
    """
    if(impurity_measure == 'entropy'):
        split_value, col, left_child, right_child = best_info_gain_entropy(parent_node.data)
    elif(impurity_measure == 'gini'):
        split_value, col, left_child, right_child = best_info_gain_gi(parent_node.data)
    else:
        print("Not valid impurity measure")
        return
        
    parent_node.split_value = split_value
    parent_node.col = col
    
    #If split_value == None - is the node a leaf
    if (parent_node.split_value == None):
        return
    
    #builds down to the left first
    left_node = parent_node.add_child(left_child, key='False')  
    build_tree(left_node, impurity_measure)
    
    #then go to the right
    right_node = parent_node.add_child(right_child, key='True')
    build_tree(right_node, impurity_measure)
    
    return

In [ ]:
def convert_to_numbers(data): 
    """
    Doc.
    
    Title: convert_to_numbers
    
    :param data: is a collection of data and the labels(target), seperated in categories
    :type data: pandas dataframe
    
    :return: data translated to numbers
    :return type: 2D array
    """
    for column in data.columns: 
        data[column] = pd.factorize(data[column])[0]
    data = data.values #numpy array 
    
    return data

In [ ]:

def learn(X, y, impurity_measure='entropy', pruning=None, feature_names=False):
    """
    Doc.

    Title: learn

    This function learns a decision tree classifier from X and y

    :param X: all data in a dataset
    :type X: 2D array

    :param y: all the labels in a dataset
    :type y: 2D array

    :param impurity_measure: defines which impurity measure to use. Entorpy or gini index
    :type impurity_measure: string

    :param pruning: a set of data used to pruning - default = Node
    :type pruning: pandas dataframe

    :param feature_names: choose to show tree with feauters or not
    :type feature_names: bool

    :return root: returns the root of the tree
    :return type: Tree
    """
    
    X_all = pd.concat((X, y.T), axis=1)
    X_all = convert_to_numbers(X_all)
    
    
    root = Tree(X_all)
    build_tree(root, impurity_measure)
    
    
    if pruning is not None:
        prun = convert_to_numbers(X_prun)
        do_pruning(root, root, prun)
        
    if feature_names:
        print_tree(root)
    
    return root


In [ ]:
def predict (x, tree):
    """
    Doc.

    Title: predict

    Predict the label(target) of a set of data

    :param x: the set of data you are supposed to predict
    :type x: array

    :param tree: the trained decision tree you use to get the prediction
    :type tree: Tree

    :return: the predicted laple
    :return type: depends on label type, number(int, float) or string
    """
    node = tree
    leaf = False
    
    #As long as node isn't a leaf, go to the correct children
    while not leaf:
        if is_number(x[node.col]):
            node = node.children[str(int(x[node.col])>=int(node.split_value))]
        else:
            node = node.children[str(x[node.col]== node.split_value)]
        leaf = node.is_leaf()
    label=get_label(node.data)
    return (label)

# Exercise 3

In [ ]:
def error_rate(data, root):
    """
    Doc.
    
    Title: errer_rate
    
    Runs a test dataset through the tree and returns number of mistaken predictions
    
    :param data: a testing dataset
    :type data: 2D array
    
    :param root: the tree you run the testdata on
    :type root: Tree
    
    :return: the total error after predicting the whole dataset
    :return type: int
    """
    
    error_total = 0
    for row in data:
        pred_label = predict(row, root)
        
        if pred_label != float(row[-1]):
            error_total += 1

    return error_total


In [ ]:
def do_pruning(temp_root, node, X_pruning): 
    """
    Doc.
    
    Title: do_pruning
    
    A recursive function that can help to avoid overfitting
    
    :param temp_root: holds the root element(node) the whole time
    :type temp_root: Tree
    
    :param node: current node , used to check if any leafs should be deleted
    :type node: Tree

    :param X_pruning: the dataset used to predict and find the error rate
    :type X_pruning: 2D array
    
    :return: no return param here, the pruning will happen when running the function 
    """
    
    if not node.is_leaf():
        do_pruning(temp_root, node.children['False'], X_pruning)
        do_pruning(temp_root, node.children['True'], X_pruning)
    else:
        return
    #if both children is leaf
    if node.children['False'].is_leaf() and node.children['True'].is_leaf():
        children_temp = node.children
        prev_error = error_rate(X_pruning, temp_root)
        node.children = None
        new_error = error_rate(X_pruning, temp_root)
        if(new_error<=prev_error):
            return
        else:
            node.children=children_temp
            return
    else:
        return

In [ ]:
def print_tree(node):
    """
    Doc. 
    
    Title: print_tree
    
    Prints out the steps in the tree (recommend to look at this step by step and paint it)

    :param node: the current node
    :type node: Tree

    :return: no return value
    """
    print (node.data)
    if node.is_leaf():
        print ('Leaf ↑')
        print ('Label: ' + str(get_label(node.data)))
        print ('')
    else:
        if is_number(node.split_value):
            print ('Split value: >='+str(node.split_value)+ " in col: " + str(node.col))
        else:
            print ('Split value: =='+node.split_value)
    print ('')      
    if not node.is_leaf():
        print('Left child (False)   ↙')
        print_tree(node.children['False'])
        print('↘  Rightchild (True)')
        print_tree(node.children['True'])
    else:
        return

# Exercise 4

In [ ]:
root = learn(X_train, y_train)
X_performance = convert_to_numbers(X_test)
error_performance= error_rate(X_performance, root)
print ('Number of errors with: impurity_measure=entropy, pruning=None: ' + str(error_performance))

In [ ]:
root = learn(X_train, y_train, impurity_measure='gini')
error_performance= error_rate(X_performance, root)
print ('Number of errors with: impurity_measure=gini index, pruning=None: ' + str(error_performance))

In [ ]:
root = learn(X_train, y_train, pruning=X_prun)
error_performance= error_rate(X_performance, root)
print ('Number of errors with: impurity_measure=entropy, pruning=Yes: ' + str(error_performance))

In [ ]:
root = learn(X_train, y_train, impurity_measure='gini', pruning=X_prun)
error_performance= error_rate(X_performance, root)
print ('Number of errors with: impurity_measure=gini index, pruning=Yes: ' + str(error_performance))

# Exercise 5

In [ ]:
root = learn(X_train, y_train, pruning=X_prun)

In [ ]:
test_X_test = convert_to_numbers(X_test)
error_test= error_rate(test_X_test, root)
print ("Accuracy with test data, with my test: " + str(100-(100*(error_test/len(X_test)))) + "%")

test_X_prun = convert_to_numbers(X_prun)
error_prun = error_rate(test_X_prun, root)
print ("Accuracy with pruning data, with my test: " + str(100-(100*(error_prun/len(X_prun))))+ "%")

In [ ]:
from sklearn import tree
clf = tree.DecisionTreeClassifier('entropy')
sk_X_train = pd.concat((X_train, y_train.T), axis=1)
sk_X_train = convert_to_numbers(sk_X_train)
sk_y_train = sk_X_train[:,-1]
sk_X_train =sk_X_train[:,:-1]


clf.fit(sk_X_train, sk_y_train)

y_sk_test = X_test.iloc[:,-1]
X_sk_test = X_test.iloc[:,:-1]
y_sk_test = y_sk_test.values
X_sk_test = X_sk_test.values


score_test= clf.score(X=X_sk_test, y=y_sk_test)


X_sk_prun = convert_to_numbers(X_prun)
y_sk_prun = X_sk_prun[:,-1]
X_sk_prun = X_sk_prun[:,:-1]

print("Accuracy with test data, with sklearn: " + str(100*score_test))

score_prun = clf.score(X=X_sk_prun, y=y_sk_prun)
print("Accuracy with pruning data, with sklearn: " + str(100*score_prun))